Нуралиев Расим. Группа ПИ19-4

**Вариант 16**
## Task 6
Задание 6. Задача об оптимальном использовании ресурсов [1,2 – гл. 3, 4]
Для производства трех видов изделий (A, B, C) используются ресурсы типа I и II,
причем закупки ресурсов ограничены возможностями поставщиков. Нормы расхода ресурсов и их запасы приведены в таблице.
1. Постройте математическую модель задачи.
2. Определите такой план производства, при котором стоимость произведенного товара из имеющихся ресурсов является наибольшей.
3. Постройте задачу, двойственную к данной.
4. Найдите решение двойственной задачи. Поясните смысл двойственных переменных.
5. Проверьте результаты вычислений на компьютере.

a = 3 <br>
b = 1 <br>
c = 2

### Построение математической модели задачи

Кол-во параметров для оптимизации: 3

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gb

from scipy.optimize import minimize, LinearConstraint, Bounds, linprog
import sympy as sym

In [36]:
a = 3 
b = 1 
c = 22

**F(x1,x2,x3)** = (6b + 12)\*x1 + (5b + 22)\*x2 + c\*x3 → max<br>

Вариант 16

$F(x1,x2,x3) = 18 * x1 + 27 * x2 + 22 * x3 $


Ограничения:
<br>
$x1 \geq 0 $ <br>
$x2 \geq 0 $ <br>
$x3 \geq 0 $ <br>
$x1 + 3*x2 + a*x3 \leq 3000 $ <br>
$6*x1 + 5*x2 + 2*x3 \leq 3320 $ <br>


In [14]:
!pip install gurobipy

In [37]:
#Вводим границы

bounds = np.array([(0, np.inf), (0, np.inf), (0, np.inf)])

#Вводим ограничения

#A_ub - матрица коэф. при x1,x2,x3 в ограничениях, A_ub @ x <= b_ub
#c0 - коэффициенты перед переменными в целевой функции
A_ub = np.array([[1, 3, a], [6, 5, 2]])
B = np.array([3000, 3320])
c0 = -np.array([6*b + 12, 5*b + 22, c])

res = linprog(c0, A_ub, B, bounds=bounds, method='simplex', options={'disp': True})

x1 = res.x[0]
x2 = res.x[1]
x3 = res.x[2]
max_value = -res.fun

print(f"Подобранные значения:\n\nМаксимальное значение: {max_value}\nx1 = {x1}\nx2 = {x2}\nx3 = {x3}")

Optimization terminated successfully.
         Current function value: -24640.000000
         Iterations: 3
Подобранные значения:

Максимальное значение: 24640.0
x1 = 247.5
x2 = 0.0
x3 = 917.5


Проверка результатов вычисления

In [31]:
b_opt = np.array([x1 + 3*x2 + a*x3, 6*x1 + 5*x2 + 2*x3])


In [32]:
assert b_opt[0] == 3000
assert  b_opt[1] == 3320

Дефицит (исчерпанность ресурсов)

In [41]:
abs(B - b_opt)

array([0., 0.])

Оценка устойчивости коэффициентов целевой функции
Минимальные и максимальные значения коэффициентов в целевой функции, сохраняющие оптимальность


In [42]:
from gurobipy import *

In [46]:
var_names = ['x1', 'x2', 'x3']                         
profit = np.array([6*b + 12, 5*b + 22, c])              
res_used = np.array([[1, 3, a], [6, 5, 2]]) 
res_avail = np.array([3000, 3320])          

n = len(profit)                                    
m = len(res_avail) 

# Инициализация модели
model = Model()

x = model.addVars(n, name=var_names) # заводим переменные в модель
obj = model.setObjective(quicksum(profit[j] * x[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(res_used[i, j] * x[j] for j in range(n)) <= res_avail[i], name=var_names[i]))
model.optimize()


# Результаты вычислений
print("X =", model.X, "\nFmax =", model.ObjVal, "\nОптимальный план двойственной задачи Pi → (y1,y2) =", model.Pi) 
print("x1 =", x[0].X)
print("Pi1 =", con[0].Pi)

# Оценка устойчивости коэффициентов целевой функции
# Минимальные и максимальные значения коэффициентов в целевой функции, сохраняющие оптимальность
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xf7e02ca1
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 3e+03]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7000000e+31   3.375000e+30   6.700000e+01      0s
       2    2.4640000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.464000000e+04
X = [247.5, 0.0, 917.5] 
Fmax = 24640.0 
Оптимальный план двойственной задачи Pi → (y1,y2) = [6.0, 2.0000000000000004]
x1 = 247.5
Pi1 = 6.0

    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
   

In [48]:
#Решение двойственной к M задачи
print('Анализ чувствительности\nFmax =', model.ObjVal)
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])

Анализ чувствительности
Fmax = 24640.0

    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
          x1        247.5            0            0         -inf        247.5          inf        247.5          inf 
          x2            0           -1            0         -inf          440          inf            0          inf 
          x3        917.5            0            0         -inf        917.5          inf        917.5          inf 


In [49]:
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp'])


  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
          x1            <            0            6         3000      553.333         4980 
          x2            <            0            2         3320         2000        18000 
